In [1]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [2]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font = 'Microsoft YaHei',font_scale = 1.3)

from pyecharts import options as opts
from pyecharts.charts import Bar,Map,WordCloud,Grid,Pie
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType

from warnings import filterwarnings
filterwarnings('ignore')

**获取数据**

In [3]:
data = pd.read_excel(r'2021年世界五百强排行榜.xlsx')
data.shape

(500, 13)

**查看是否有缺失值**

In [4]:
data.isnull().sum().sum()

6

**处理缺失值**

In [5]:
def na_check(df):
    if df.isna().sum().sum() == 0:
        return "无缺失值"
    else:
        na_num = df.isna().sum().sum()
        df.dropna(axis = 0,inplace = True)
        return '含有缺失值' + str(na_num) + "个缺失值"

In [6]:
na_check(data)

'含有缺失值6个缺失值'

**数据类型转换**

In [7]:
data['营业收入(百万美元)'] = data['营业收入(百万美元)'].str.replace(',','').astype('float')

In [8]:
data.head()

,排名,公司名称(中文),公司名称(中文)链接,营业收入(百万美元),利润(百万美元),国家,关键数据,详情标题,字段1,字段2,字段3,字段4,字段5
0,1,沃尔玛（WALMART),http://www.fortunechina.com/global500/3/2021,559151.0,"13,510",美国,+,沃尔玛WALMART,美国 (U.S.),本顿维尔 (Bentonville),C. Douglas McMillon,2300000.0,www.stock.walmart.com
1,2,国家电网有限公司（STATE GRID),http://www.fortunechina.com/global500/15/2021,386617.7,"5,580.4",中国,+,国家电网有限公司STATE GRID,中国 (China),北京 (Beijing),Xin Baoan,896360.0,www.sgcc.com.cn
2,3,亚马逊（AMAZON.COM),http://www.fortunechina.com/global500/485/2021,386064.0,"21,331",美国,+,亚马逊AMAZON.COM,美国 (U.S.),西雅图 (Seattle),Andrew R. Jassy,1298000.0,www.amazon.com
3,4,中国石油天然气集团有限公司（CHINA NATIONAL PETROLEUM),http://www.fortunechina.com/global500/13/2021,283957.6,"4,575.2",中国,+,中国石油天然气集团有限公司CHINA NATIONAL PETROLEUM,中国 (China),北京 (Beijing),Li FanRong,1242245.0,www.cnpc.com.cn
4,5,中国石油化工集团有限公司（SINOPEC GROUP),http://www.fortunechina.com/global500/9/2021,283727.6,"6,205.2",中国,+,中国石油化工集团有限公司SINOPEC GROUP,中国 (China),北京 (Beijing),Zhang Yuzhuo,553833.0,www.sinopec.com


**世界地图中英文名称映射表**

In [21]:
nameMap = {
        'Singapore Rep.':'新加坡',
        'Dominican Rep.':'多米尼加',
        'Palestine':'巴勒斯坦',
        'Bahamas':'巴哈马',
        'Timor-Leste':'东帝汶',
        'Afghanistan':'阿富汗',
        'Guinea-Bissau':'几内亚比绍',
        "Côte d'Ivoire":'科特迪瓦',
        'Siachen Glacier':'锡亚琴冰川',
        "Br. Indian Ocean Ter.":'英属印度洋领土',
        'Angola':'安哥拉',
        'Albania':'阿尔巴尼亚',
        'United Arab Emirates':'阿联酋',
        'Argentina':'阿根廷',
        'Armenia':'亚美尼亚',
        'French Southern and Antarctic Lands':'法属南半球和南极领地',
        'Australia':'澳大利亚',
        'Austria':'奥地利',
        'Azerbaijan':'阿塞拜疆',
        'Burundi':'布隆迪',
        'Belgium':'比利时',
        'Benin':'贝宁',
        'Burkina Faso':'布基纳法索',
        'Bangladesh':'孟加拉国',
        'Bulgaria':'保加利亚',
        'The Bahamas':'巴哈马',
        'Bosnia and Herz.':'波斯尼亚和黑塞哥维那',
        'Belarus':'白俄罗斯',
        'Belize':'伯利兹',
        'Bermuda':'百慕大',
        'Bolivia':'玻利维亚',
        'Brazil':'巴西',
        'Brunei':'文莱',
        'Bhutan':'不丹',
        'Botswana':'博茨瓦纳',
        'Central African Rep.':'中非',
        'Canada':'加拿大',
        'Switzerland':'瑞士',
        'Chile':'智利',
        'China':'中国',
        'Ivory Coast':'象牙海岸',
        'Cameroon':'喀麦隆',
        'Dem. Rep. Congo':'刚果民主共和国',
        'Congo':'刚果',
        'Colombia':'哥伦比亚',
        'Costa Rica':'哥斯达黎加',
        'Cuba':'古巴',
        'N. Cyprus':'北塞浦路斯',
        'Cyprus':'塞浦路斯',
        'Czech Rep.':'捷克',
        'Germany':'德国',
        'Djibouti':'吉布提',
        'Denmark':'丹麦',
        'Algeria':'阿尔及利亚',
        'Ecuador':'厄瓜多尔',
        'Egypt':'埃及',
        'Eritrea':'厄立特里亚',
        'Spain':'西班牙',
        'Estonia':'爱沙尼亚',
        'Ethiopia':'埃塞俄比亚',
        'Finland':'芬兰',
        'Fiji':'斐',
        'Falkland Islands':'福克兰群岛',
        'France':'法国',
        'Gabon':'加蓬',
        'United Kingdom':'英国',
        'Georgia':'格鲁吉亚',
        'Ghana':'加纳',
        'Guinea':'几内亚',
        'Gambia':'冈比亚',
        'Guinea Bissau':'几内亚比绍',
        'Eq. Guinea':'赤道几内亚',
        'Greece':'希腊',
        'Greenland':'格陵兰',
        'Guatemala':'危地马拉',
        'French Guiana':'法属圭亚那',
        'Guyana':'圭亚那',
        'Honduras':'洪都拉斯',
        'Croatia':'克罗地亚',
        'Haiti':'海地',
        'Hungary':'匈牙利',
        'Indonesia':'印度尼西亚',
        'India':'印度',
        'Ireland':'爱尔兰',
        'Iran':'伊朗',
        'Iraq':'伊拉克',
        'Iceland':'冰岛',
        'Israel':'以色列',
        'Italy':'意大利',
        'Jamaica':'牙买加',
        'Jordan':'约旦',
        'Japan':'日本',
        'Japan':'日本本土',
        'Kazakhstan':'哈萨克斯坦',
        'Kenya':'肯尼亚',
        'Kyrgyzstan':'吉尔吉斯斯坦',
        'Cambodia':'柬埔寨',
        'Korea':'韩国',
        'Kosovo':'科索沃',
        'Kuwait':'科威特',
        'Lao PDR':'老挝',
        'Lebanon':'黎巴嫩',
        'Liberia':'利比里亚',
        'Libya':'利比亚',
        'Sri Lanka':'斯里兰卡',
        'Lesotho':'莱索托',
        'Lithuania':'立陶宛',
        'Luxembourg':'卢森堡',
        'Latvia':'拉脱维亚',
        'Morocco':'摩洛哥',
        'Moldova':'摩尔多瓦',
        'Madagascar':'马达加斯加',
        'Mexico':'墨西哥',
        'Macedonia':'马其顿',
        'Mali':'马里',
        'Myanmar':'缅甸',
        'Montenegro':'黑山',
        'Mongolia':'蒙古',
        'Mozambique':'莫桑比克',
        'Mauritania':'毛里塔尼亚',
        'Malawi':'马拉维',
        'Malaysia':'马来西亚',
        'Namibia':'纳米比亚',
        'New Caledonia':'新喀里多尼亚',
        'Niger':'尼日尔',
        'Nigeria':'尼日利亚',
        'Nicaragua':'尼加拉瓜',
        'Netherlands':'荷兰',
        'Norway':'挪威',
        'Nepal':'尼泊尔',
        'New Zealand':'新西兰',
        'Oman':'阿曼',
        'Pakistan':'巴基斯坦',
        'Panama':'巴拿马',
        'Peru':'秘鲁',
        'Philippines':'菲律宾',
        'Papua New Guinea':'巴布亚新几内亚',
        'Poland':'波兰',
        'Puerto Rico':'波多黎各',
        'Dem. Rep. Korea':'朝鲜',
        'Portugal':'葡萄牙',
        'Paraguay':'巴拉圭',
        'Qatar':'卡塔尔',
        'Romania':'罗马尼亚',
        'Russia':'俄罗斯',
        'Rwanda':'卢旺达',
        'W. Sahara':'西撒哈拉',
        'Saudi Arabia':'沙特阿拉伯',
        'Sudan':'苏丹',
        'S. Sudan':'南苏丹',
        'Senegal':'塞内加尔',
        'Solomon Is.':'所罗门群岛',
        'Sierra Leone':'塞拉利昂',
        'El Salvador':'萨尔瓦多',
        'Somaliland':'索马里兰',
        'Somalia':'索马里',
        'Serbia':'塞尔维亚',
        'Suriname':'苏里南',
        'Slovakia':'斯洛伐克',
        'Slovenia':'斯洛文尼亚',
        'Sweden':'瑞典',
        'Swaziland':'斯威士兰',
        'Syria':'叙利亚',
        'Chad':'乍得',
        'Togo':'多哥',
        'Thailand':'泰国',
        'Tajikistan':'塔吉克斯坦',
        'Turkmenistan':'土库曼斯坦',
        'East Timor':'东帝汶',
        'Trinidad and Tobago':'特里尼达和多巴哥',
        'Tunisia':'突尼斯',
        'Turkey':'土耳其',
        'Tanzania':'坦桑尼亚',
        'Uganda':'乌干达',
        'Ukraine':'乌克兰',
        'Uruguay':'乌拉圭',
        'United States':'美国',
        'Uzbekistan':'乌兹别克斯坦',
        'Venezuela':'委内瑞拉',
        'Vietnam':'越南',
        'Vanuatu':'瓦努阿图',
        'West Bank':'西岸',
        'Yemen':'也门',
        'South Africa':'南非',
        'Zambia':'赞比亚',
        'Zimbabwe':'津巴布韦'
    }

### 1. 世界500强企业分布图

In [38]:
x_data = data['国家'].value_counts().index.tolist()
y_data = data['国家'].value_counts().values.tolist()

map1 = (
    Map()
    .add(
        '',
        [list(z) for z in zip(x_data, y_data)],
        maptype = 'world',is_map_symbol_show = False,
        name_map = nameMap
        )
    .set_series_opts(label_opts = opts.LabelOpts(is_show = False))
    .set_global_opts(title_opts = opts.TitleOpts(title = '世界500强企业'),
                     visualmap_opts = opts.VisualMapOpts(max_ = data['国家'].value_counts().max(),min_ = data['国家'].value_counts().min(),
                                                       is_piecewise=False ))
)
map1.load_javascript()

In [39]:
map1.render_notebook()

### 2. 2021年各国进入500强的企业数量排名

In [42]:
result = data['国家'].value_counts().sort_values()   # 计数，并且按照倒序排列

In [43]:
x_data = result.index.tolist()

In [44]:
y_data = result.values.tolist()

In [45]:
bar = (
    Bar(init_opts=opts.InitOpts(width = '900px',height = '900px'))
    .add_xaxis(x_data)
    .add_yaxis('',y_data)
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position="right"))
    .set_global_opts(title_opts=opts.TitleOpts(title="2021年各国进入500强企业数量排名"))
)

bar.load_javascript()

In [46]:
bar.render_notebook()

### 3. 中国各城市进入500强企业数量排名

In [29]:
data.head()

,排名,公司名称(中文),公司名称(中文)链接,营业收入(百万美元),利润(百万美元),国家,关键数据,详情标题,字段1,字段2,字段3,字段4,字段5
0,1,沃尔玛（WALMART),http://www.fortunechina.com/global500/3/2021,559151.0,"13,510",美国,+,沃尔玛WALMART,美国 (U.S.),本顿维尔 (Bentonville),C. Douglas McMillon,2300000.0,www.stock.walmart.com
1,2,国家电网有限公司（STATE GRID),http://www.fortunechina.com/global500/15/2021,386617.7,"5,580.4",中国,+,国家电网有限公司STATE GRID,中国 (China),北京 (Beijing),Xin Baoan,896360.0,www.sgcc.com.cn
2,3,亚马逊（AMAZON.COM),http://www.fortunechina.com/global500/485/2021,386064.0,"21,331",美国,+,亚马逊AMAZON.COM,美国 (U.S.),西雅图 (Seattle),Andrew R. Jassy,1298000.0,www.amazon.com
3,4,中国石油天然气集团有限公司（CHINA NATIONAL PETROLEUM),http://www.fortunechina.com/global500/13/2021,283957.6,"4,575.2",中国,+,中国石油天然气集团有限公司CHINA NATIONAL PETROLEUM,中国 (China),北京 (Beijing),Li FanRong,1242245.0,www.cnpc.com.cn
4,5,中国石油化工集团有限公司（SINOPEC GROUP),http://www.fortunechina.com/global500/9/2021,283727.6,"6,205.2",中国,+,中国石油化工集团有限公司SINOPEC GROUP,中国 (China),北京 (Beijing),Zhang Yuzhuo,553833.0,www.sinopec.com


In [51]:
data['城市'] = data['字段2'].str.split('(').str[0]

In [52]:
result2 = data[data['国家'] == '中国']

In [53]:
result2 = result2['城市'].value_counts()
result2

北京       60
上海        9
香港        9
深圳        8
杭州        7
广州        5
台北        5
厦门        3
石家庄       2
苏州市       2
福州        2
济南        2
乌鲁木齐      2
佛山        2
西安        2
桃园        1
龙岩        1
鞍山        1
芜湖        1
阳泉        1
滨州        1
新竹        1
铜陵        1
南京        1
张家港       1
金昌        1
大同        1
昆明        1
武汉        1
潍坊        1
温州        1
贵溪        1
青岛        1
珠海        1
南宁        1
太原        1
长春        1
新北        1
Name: 城市, dtype: int64

In [54]:
x_data = result2.index.tolist()
y_data = result2.values.tolist()

In [57]:
bar = Bar(init_opts=opts.InitOpts(width = '1000px',height = '500px'))
bar.add_xaxis(x_data)
bar.add_yaxis('',y_data)
# bar.reversal_axis()
# bar.set_series_opts(label_opts=opts.LabelOpts(position="right"))
bar.set_global_opts(xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=75)),title_opts=opts.TitleOpts(title="500强企业在中国各城市的数量"))
bar.load_javascript()

In [58]:
bar.render_notebook()